In [1]:
import sys, os
sys.path.append(os.path.abspath(".."))

In [ ]:
# =========================
# From-scratch WEIGHTED Regression Tree (NumPy only)
# + metrics
# + memory-light cost-complexity pruning (α) with CV
# =========================
import numpy as np
from dataclasses import dataclass
from typing import Optional, Tuple, List

# -------- Metrics --------
def wmae(y, yhat, w):
    w = np.asarray(w, float)
    return (np.abs(y - yhat) * w).sum() / w.sum()

def wrmse(y, yhat, w):
    w = np.asarray(w, float)
    return np.sqrt(((y - yhat)**2 * w).sum() / w.sum())

def weighted_r2(y, yhat, w):
    w = np.asarray(w, float)
    ybar = (y * w).sum() / w.sum()
    ss_res = ((y - yhat)**2 * w).sum()
    ss_tot = ((y - ybar)**2 * w).sum()
    return 1.0 - ss_res / ss_tot if ss_tot > 0 else 0.0

def poisson_deviance(y_counts, exposure, yhat_rate):
    y = np.asarray(y_counts, float)
    exp = np.asarray(exposure, float)
    lam = np.clip(yhat_rate, 1e-12, None) * exp
    term = np.where(y > 0, y * np.log(y / lam), 0.0) - (y - lam)
    return 2.0 * term.sum()

# -------- Helpers --------
def wmean(y, w):
    sw = w.sum()
    return (y * w).sum() / sw if sw > 0 else 0.0

def leaf_sse_from_stats(sw, swy, swy2):
    # SSE = sum w(y - mean)^2 = sum(wy^2) - (sum(wy))^2 / sum(w)
    return float(swy2 - (swy * swy) / sw) if sw > 0 else 0.0

@dataclass
class Node:
    feature: Optional[int] = None
    threshold: Optional[float] = None
    left: Optional["Node"] = None
    right: Optional["Node"] = None
    value: Optional[float] = None   # leaf prediction (weighted mean)

    # sufficient stats for THIS NODE'S SAMPLE SET (fixed after fit)
    sw: float = 0.0                  # sum of weights
    swy: float = 0.0                 # sum of w*y
    swy2: float = 0.0                # sum of w*y^2

    # subtree stats (change as we prune)
    subtree_sse: float = 0.0
    subtree_leaves: int = 1

    def is_leaf(self) -> bool:
        return self.value is not None

class DecisionTreeRegressorScratch:
    """
    Exposure-weighted regression tree for claim rate.
    - Split criterion: minimize weighted SSE.
    - Leaf value: exposure-weighted mean.
    - Pre-pruning: max_depth, min_leaf_weight (exposure units).
    Uses only NumPy for training/prediction.
    """
    def __init__(self, max_depth: Optional[int] = None, min_leaf_weight: float = 5.0):
        self.max_depth = max_depth
        self.min_leaf_weight = float(min_leaf_weight)
        self.root: Optional[Node] = None

    def fit(self, X: np.ndarray, y: np.ndarray, w: np.ndarray):
        X = np.asarray(X, float)
        y = np.asarray(y, float)
        w = np.asarray(w, float)
        idx = np.arange(X.shape[0], dtype=int)
        self.root = self._build_tree(X, y, w, idx, depth=0)
        # initialize subtree stats once
        _update_subtree_stats(self.root)
        return self

    def predict(self, X: np.ndarray) -> np.ndarray:
        X = np.asarray(X, float)
        return np.array([self._traverse(x, self.root) for x in X], float)

    # ----- internal: build tree -----
    def _build_tree(self, X, y, w, idx, depth) -> Node:
        sw = w[idx].sum()
        swy = (y[idx] * w[idx]).sum()
        swy2 = ((y[idx] ** 2) * w[idx]).sum()

        # stopping rules
        if (self.max_depth is not None and depth >= self.max_depth) or \
           (sw < 2 * self.min_leaf_weight) or \
           np.allclose(y[idx], y[idx][0]):
            return Node(value=(swy / sw if sw > 0 else 0.0), sw=sw, swy=swy, swy2=swy2,
                        subtree_sse=leaf_sse_from_stats(sw, swy, swy2), subtree_leaves=1)

        feat, thr, L, R = self._best_split(X, y, w, idx)
        if feat is None:
            return Node(value=(swy / sw if sw > 0 else 0.0), sw=sw, swy=swy, swy2=swy2,
                        subtree_sse=leaf_sse_from_stats(sw, swy, swy2), subtree_leaves=1)

        left  = self._build_tree(X, y, w, L, depth+1)
        right = self._build_tree(X, y, w, R, depth+1)
        return Node(feature=feat, threshold=thr, left=left, right=right,
                    value=None, sw=sw, swy=swy, swy2=swy2)

    # ----- internal: best split -----
    def _best_split(self, X, y, w, idx) -> Tuple[Optional[int], Optional[float], Optional[np.ndarray], Optional[np.ndarray]]:
        best = (None, None, None, None, np.inf)
        n, d = X.shape
        for j in range(d):
            xj = X[idx, j]
            uniq = np.unique(xj)
            if uniq.size <= 1:
                continue
            # thresholds: midpoints; for one-hot 0/1, just 0.5
            if uniq.size == 2 and uniq.min() == 0.0 and uniq.max() == 1.0:
                candidates = [0.5]
            else:
                u = np.unique(np.sort(xj))
                candidates = (u[:-1] + u[1:]) / 2.0

            for t in candidates:
                Lmask = xj <= t
                if not Lmask.any() or Lmask.all():
                    continue
                L = idx[Lmask]; R = idx[~Lmask]

                swL = w[L].sum(); swR = w[R].sum()
                if swL < self.min_leaf_weight or swR < self.min_leaf_weight:
                    continue

                swyL = (y[L] * w[L]).sum(); swyR = (y[R] * w[R]).sum()
                swy2L = ((y[L]**2) * w[L]).sum(); swy2R = ((y[R]**2) * w[R]).sum()
                sseL = leaf_sse_from_stats(swL, swyL, swy2L)
                sseR = leaf_sse_from_stats(swR, swyR, swy2R)
                sse  = sseL + sseR
                if sse < best[4]:
                    best = (j, t, L, R, sse)

        return best[0], best[1], best[2], best[3]

    # ----- internal: predict one -----
    def _traverse(self, x, node: Node) -> float:
        while node.value is None:
            node = node.left if x[node.feature] <= node.threshold else node.right
        return node.value

# ---- subtree stat maintenance (no indices stored) ----
def _update_subtree_stats(node: Node) -> Tuple[float, int]:
    if node.value is not None:
        node.subtree_sse = leaf_sse_from_stats(node.sw, node.swy, node.swy2)
        node.subtree_leaves = 1
        return node.subtree_sse, 1
    sL, lL = _update_subtree_stats(node.left)
    sR, lR = _update_subtree_stats(node.right)
    node.subtree_sse = sL + sR
    node.subtree_leaves = lL + lR
    return node.subtree_sse, node.subtree_leaves

def _alpha_of(node: Node) -> float:
    if node.value is not None:
        return np.inf
    sse_leaf = leaf_sse_from_stats(node.sw, node.swy, node.swy2)
    denom = max(node.subtree_leaves - 1, 1e-12)
    return (sse_leaf - node.subtree_sse) / denom

def _collect_internal(node: Node, acc: List[Node]):
    if node is None or node.value is not None:
        return
    acc.append(node)
    _collect_internal(node.left, acc)
    _collect_internal(node.right, acc)

def _prune_once_inplace(root: Node) -> float:
    """Prune all nodes with minimal α (weakest-link). Returns α*."""
    _update_subtree_stats(root)
    internals: List[Node] = []
    _collect_internal(root, internals)
    if not internals:
        return np.inf
    alphas = np.array([_alpha_of(n) for n in internals])
    a_star = float(alphas.min())
    tol = 1e-12

    def prune_mark(n: Node):
        if n.value is not None:
            return
        a = _alpha_of(n)
        if abs(a - a_star) <= tol:
            # turn node into a LEAF using its fixed stats
            mu = (n.swy / n.sw) if n.sw > 0 else 0.0
            n.feature = n.threshold = None
            n.left = n.right = None
            n.value = float(mu)
            n.subtree_sse = leaf_sse_from_stats(n.sw, n.swy, n.swy2)
            n.subtree_leaves = 1
        else:
            prune_mark(n.left); prune_mark(n.right)

    prune_mark(root)
    _update_subtree_stats(root)
    return a_star

def pruning_path_stream(root: Node):
    """
    Generator yielding (alpha_star, leaves_now) while pruning in place.
    Starts from current tree; first yield is alpha to prune to next simpler tree.
    """
    while True:
        a = _prune_once_inplace(root)
        yield a, root.subtree_leaves
        if root.subtree_leaves == 1 or not np.isfinite(a):
            break

def predict_with_root(X: np.ndarray, root: Node) -> np.ndarray:
    X = np.asarray(X, float)
    def _one(x, node):
        while node.value is None:
            node = node.left if x[node.feature] <= node.threshold else node.right
        return node.value
    return np.array([_one(X[i], root) for i in range(X.shape[0])], float)

def kfold_indices(n_samples: int, n_splits: int, seed=42, shuffle=True):
    rng = np.random.default_rng(seed)
    idx = np.arange(n_samples, dtype=int)
    if shuffle: rng.shuffle(idx)
    folds = np.array_split(idx, n_splits)
    for i in range(n_splits):
        val = folds[i]
        tr  = np.concatenate([folds[j] for j in range(n_splits) if j != i])
        yield tr, val

def cv_select_alpha_memorylight(X, y, w, base_params, kfold=5, seed=7):
    """
    Memory-light α selection:
    1) Build base tree on ALL data; stream its pruning alphas into a global α grid.
    2) For each fold, rebuild base tree on train; stream pruning; record (alpha, score) pairs.
    3) For each global α, use the fold's first α' >= α (right-bin) score.
    """
    # Global α grid
    base_all = DecisionTreeRegressorScratch(**base_params).fit(X, y, w)
    global_alphas = []
    # make a fresh copy by re-fitting (cheaper than deep copy)
    root_all = DecisionTreeRegressorScratch(**base_params).fit(X, y, w).root
    for a, _ in pruning_path_stream(root_all):
        if np.isfinite(a):
            global_alphas.append(a)
        if getattr(root_all, "subtree_leaves", 1) == 1:
            break
    global_alphas = np.array(global_alphas, float)
    if global_alphas.size == 0:
        return {"alpha": 0.0, "cv_wmae": wmae(y, predict_with_root(X, base_all.root), w),
                "alphas": [], "cv_curve": np.array([0.0])}

    # Per-fold streamed curves (α_folds, score_folds)
    fold_alphas = []
    fold_scores = []
    for tr, va in kfold_indices(len(y), kfold, seed=seed):
        base = DecisionTreeRegressorScratch(**base_params).fit(X[tr], y[tr], w[tr])
        a_list = []
        s_list = []
        # evaluate current (unpruned) too: treat as alpha=0 (optional)
        yhat0 = predict_with_root(X[va], base.root)
        s_list.append(wmae(y[va], yhat0, w[va]))
        a_list.append(0.0)
        # stream pruning
        for a, _ in pruning_path_stream(base.root):
            if not np.isfinite(a):  # finished
                break
            yhat = predict_with_root(X[va], base.root)
            s_list.append(wmae(y[va], yhat, w[va]))
            a_list.append(a)
            if getattr(base.root, "subtree_leaves", 1) == 1:
                break
        fold_alphas.append(np.array(a_list, float))
        fold_scores.append(np.array(s_list, float))

    # Aggregate on the global α grid by right-binning each fold's curve
    mean_scores = []
    for A in global_alphas:
        acc = []
        for a_arr, s_arr in zip(fold_alphas, fold_scores):
            # pick first index where a_arr >= A; if none, use last
            j = int(np.searchsorted(a_arr, A, side="left"))
            if j >= len(s_arr): j = len(s_arr) - 1
            acc.append(s_arr[j])
        mean_scores.append(np.mean(acc))
    mean_scores = np.array(mean_scores, float)
    kbest = int(np.argmin(mean_scores))
    return {"alpha": float(global_alphas[kbest]),
            "cv_wmae": float(mean_scores[kbest]),
            "alphas": global_alphas,
            "cv_curve": mean_scores}


In [ ]:
# =============================
# Load + preprocess + tiny pre-pruning CV
# =============================
import pandas as pd
from preprocessing.preprocessing_utils import preprocess_for_tree  # your function

# Load
train = pd.read_csv("../data/claims_train.csv")
test  = pd.read_csv("../data/claims_test.csv")

# Preprocess (no scaling; OHE cats; returns rate & exposure)
X_tr, y_tr_rate, w_tr = preprocess_for_tree(train)
X_te, y_te_rate, w_te = preprocess_for_tree(test)

# Reconstruct counts for Poisson deviance
y_tr_cnt = (y_tr_rate * w_tr).to_numpy(float)
y_te_cnt = (y_te_rate * w_te).to_numpy(float)

# To NumPy (OPTIONAL: float32 to save memory)
X_np = X_tr.values.astype(np.float32)
y_np = y_tr_rate.values.astype(np.float32)
w_np = w_tr.values.astype(np.float32)

# Simple hold-out split for quick validation in the grid
rng = np.random.default_rng(42)
idx = np.arange(len(y_np))
rng.shuffle(idx)
cut = int(0.8 * len(idx))
tr_idx, va_idx = idx[:cut], idx[cut:]

X_tr_np, y_tr_np, w_tr_np = X_np[tr_idx], y_np[tr_idx], w_np[tr_idx]
X_va_np, y_va_np, w_va_np = X_np[va_idx], y_np[va_idx], w_np[va_idx]
y_va_cnt_np = y_tr_cnt[va_idx]  # counts for Poisson deviance

# ---- Tiny grid over pre-pruning caps (selection by WMAE) ----
depth_grid = [5,10,25,50,100]
leafw_grid = [10,15,20,25,30,50]

best = None
for d in depth_grid:
    for m in leafw_grid:
        tree = DecisionTreeRegressorScratch(max_depth=d, min_leaf_weight=m).fit(X_tr_np, y_tr_np, w_tr_np)
        yhat_va = tree.predict(X_va_np)
        score = wmae(y_va_np, yhat_va, w_va_np)
        if (best is None) or (score < best["wmae"]):
            best = {"max_depth": d, "min_leaf_weight": m, "wmae": score, "yhat_va": yhat_va}
        print(f"Pre-pruning (max_depth={d}, min_leaf_weight={m}): WMAE={score:.6f}")

print("Chosen pre-pruning caps:", {k: best[k] for k in ["max_depth","min_leaf_weight","wmae"]})

# Report validation metrics for the chosen caps
yhat_va = best["yhat_va"]
print("\nValidation metrics (chosen caps):")
print(" WMAE         :", wmae(y_va_np, yhat_va, w_va_np))
print(" WRMSE        :", wrmse(y_va_np, yhat_va, w_va_np))
print(" Weighted R^2 :", weighted_r2(y_va_np, yhat_va, w_va_np))
print(" Poisson Dev. :", poisson_deviance(y_va_cnt_np, w_va_np, yhat_va))


Pre-pruning (max_depth=11, min_leaf_weight=9.0): WMAE=0.182040
Pre-pruning (max_depth=11, min_leaf_weight=11.0): WMAE=0.182022
Pre-pruning (max_depth=11, min_leaf_weight=13.0): WMAE=0.181984
Pre-pruning (max_depth=11, min_leaf_weight=15.0): WMAE=0.182097
Pre-pruning (max_depth=11, min_leaf_weight=20.0): WMAE=0.181914
Pre-pruning (max_depth=11, min_leaf_weight=50.0): WMAE=0.182112
Pre-pruning (max_depth=13, min_leaf_weight=9.0): WMAE=0.181675
Pre-pruning (max_depth=13, min_leaf_weight=11.0): WMAE=0.181740
Pre-pruning (max_depth=13, min_leaf_weight=13.0): WMAE=0.181659
Pre-pruning (max_depth=13, min_leaf_weight=15.0): WMAE=0.181849
Pre-pruning (max_depth=13, min_leaf_weight=20.0): WMAE=0.181738
Pre-pruning (max_depth=13, min_leaf_weight=50.0): WMAE=0.182081
Pre-pruning (max_depth=15, min_leaf_weight=9.0): WMAE=0.181912
Pre-pruning (max_depth=15, min_leaf_weight=11.0): WMAE=0.181755
Pre-pruning (max_depth=15, min_leaf_weight=13.0): WMAE=0.181817
Pre-pruning (max_depth=15, min_leaf_weight=

C:\Users\Emil\AppData\Local\Temp\ipykernel_1972\4195817464.py:30: RuntimeWarning: divide by zero encountered in log
  term = np.where(y > 0, y * np.log(y / lam), 0.0) - (y - lam)
C:\Users\Emil\AppData\Local\Temp\ipykernel_1972\4195817464.py:30: RuntimeWarning: invalid value encountered in multiply
  term = np.where(y > 0, y * np.log(y / lam), 0.0) - (y - lam)


In [5]:
# =============================
# Post-pruning via streaming CCP (α) + final eval
# =============================
base_params = {"max_depth": best["max_depth"], "min_leaf_weight": best["min_leaf_weight"]}

# Select alpha with memory-light CV
alpha_sel = cv_select_alpha_memorylight(X_np, y_np, w_np, base_params=base_params, kfold=5, seed=7)
alpha_star = alpha_sel["alpha"]
print("Chosen α:", alpha_star, "| CV WMAE:", alpha_sel["cv_wmae"])

# Fit base tree on ALL training with chosen caps
final_base = DecisionTreeRegressorScratch(**base_params).fit(X_np, y_np, w_np)

# Prune IN PLACE until we reach α >= alpha_star (streaming; no tree copies)
# (We advance pruning steps and stop once the just-applied step's α ≥ alpha_star.)
alpha_reached = 0.0
for a, _ in pruning_path_stream(final_base.root):
    alpha_reached = a
    if not np.isfinite(a) or a >= alpha_star or getattr(final_base.root, "subtree_leaves", 1) == 1:
        break
    print(f"Pruned to α={a}, leaves={final_base.root.subtree_leaves}")

# ---- Test evaluation ----
X_te_np = X_te.values.astype(np.float32)
y_te_np = y_te_rate.values.astype(np.float32)
w_te_np = w_te.values.astype(np.float32)
yhat_te = predict_with_root(X_te_np, final_base.root)

print("\nTest metrics (final pruned tree):")
print(" WMAE         :", wmae(y_te_np, yhat_te, w_te_np))
print(" WRMSE        :", wrmse(y_te_np, yhat_te, w_te_np))
print(" Weighted R^2 :", weighted_r2(y_te_np, yhat_te, w_te_np))
y_te_cnt_np = (y_te_np * w_te_np).astype(np.float32)
print(" Poisson Dev. :", poisson_deviance(y_te_cnt_np, w_te_np, yhat_te))



KeyboardInterrupt: 